# **Importing the dependencies**

In [1]:
import os
import json

from zipfile import ZipFile
import pandas as pd
# for split dataset into train data and test data using sklearn
from sklearn.model_selection import train_test_split
# for provide api and easier build neural network model
from tensorflow.keras.models import Sequential
# 3 layers are build. Dense layer is fully connected, embedding layer is the first layer of lstm network 
# and lstm
from tensorflow.keras.layers import Dense, Embedding, LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Load Data

In [2]:
data = pd.read_csv('/kaggle/input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv')

In [3]:
data.shape

(50000, 2)

In [4]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [5]:
# How many positive values and how many negative values

data["sentiment"].value_counts()

sentiment
positive    25000
negative    25000
Name: count, dtype: int64

In [9]:
# convert positive data to 1 and negative data to 0
data.replace({"sentiment":{"positive":1, "negative":0}}, inplace=True)

In [10]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [11]:
data["sentiment"].value_counts()

sentiment
1    25000
0    25000
Name: count, dtype: int64

# Split Data (Train and Test)

In [12]:
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

In [13]:
print(train_data.shape)
print(test_data.shape)

(40000, 2)
(10000, 2)


# Data Preprocessing

In [14]:
# Tokenize text data
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(train_data["review"])
x_train = pad_sequences(tokenizer.texts_to_sequences(train_data["review"]), maxlen=200)
x_test = pad_sequences(tokenizer.texts_to_sequences(test_data["review"]), maxlen=200)

In [15]:
print(x_train)

[[1935    1 1200 ...  205  351 3856]
 [   3 1651  595 ...   89  103    9]
 [   0    0    0 ...    2  710   62]
 ...
 [   0    0    0 ... 1641    2  603]
 [   0    0    0 ...  245  103  125]
 [   0    0    0 ...   70   73 2062]]


In [16]:
print(x_test)

[[   0    0    0 ...  995  719  155]
 [  12  162   59 ...  380    7    7]
 [   0    0    0 ...   50 1088   96]
 ...
 [   0    0    0 ...  125  200 3241]
 [   0    0    0 ... 1066    1 2305]
 [   0    0    0 ...    1  332   27]]


In [18]:
y_train = train_data["sentiment"]
y_test = test_data["sentiment"]

In [19]:
print(y_train)

39087    0
30893    0
45278    1
16398    0
13653    0
        ..
11284    1
44732    1
38158    0
860      1
15795    1
Name: sentiment, Length: 40000, dtype: int64


# Build Model: LSTM

In [21]:
model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=128, input_length=200))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation="sigmoid"))

In [22]:
# Compile the model
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

# Trainning the Model

In [23]:
model.fit(x_train, y_train, epochs=10, batch_size=64, validation_split=0.2)

Epoch 1/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 143s 280ms/step - accuracy: 0.7226 - loss: 0.5319 - val_accuracy: 0.8299 - val_loss: 0.3958
Epoch 2/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 140s 280ms/step - accuracy: 0.8486 - loss: 0.3673 - val_accuracy: 0.7979 - val_loss: 0.4427
Epoch 3/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 145s 289ms/step - accuracy: 0.7566 - loss: 0.5059 - val_accuracy: 0.8288 - val_loss: 0.3864
Epoch 4/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 143s 286ms/step - accuracy: 0.8378 - loss: 0.3666 - val_accuracy: 0.8709 - val_loss: 0.3143
Epoch 5/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 141s 282ms/step - accuracy: 0.8974 - loss: 0.2566 - val_accuracy: 0.8832 - val_loss: 0.2869
Epoch 6/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 140s 280ms/step - accuracy: 0.9121 - loss: 0.2254 - val_accuracy: 0.8842 - val_loss: 0.2938
Epoch 7/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 142s 279ms/step - accuracy: 0.9259 - loss: 0.1889 - val_accuracy: 0.8842 - val_loss: 0.3093
Epoch 8/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 140s 280ms/step - accuracy: 0.9403 -

# Model Evaluation

In [24]:
loss, accuracy = model.evaluate(x_test, y_test)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")

313/313 ━━━━━━━━━━━━━━━━━━━━ 25s 80ms/step - accuracy: 0.8838 - loss: 0.3248
Test Loss: 0.3233829140663147
Test Accuracy: 0.8862000107765198


# Building a Predictive System

In [25]:
def predict_sentiment(review):
#     tokenize and pad the review
    sequence = tokenizer.texts_to_sequences([review])
    padded_sequence = pad_sequences(sequence, maxlen=200)
    prediction = model.predict(padded_sequence)
    sentiment = "positive" if prediction[0][0] > 0.5 else "negative"
    return sentiment

# Example Testing

In [26]:
new_review = "This movie was fantastic. I love it."
sentiment = predict_sentiment(new_review)
print (f"The sentiment of the review is: {sentiment}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 248ms/step
The sentiment of the review is: positive


In [27]:
new_review = "This movie was not good."
sentiment = predict_sentiment(new_review)
print (f"The sentiment of the review is: {sentiment}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
The sentiment of the review is: negative
